In [1]:
# Importaciones e instalaciones necesarias
from selenium import webdriver
import random 
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager # pip install webdriver-manager

# Para trabajar con mongodb
!pip install pymongo
from pymongo import MongoClient
client = MongoClient('localhost')
db = client['pruebaTusDatos']
col = db['procesos']

In [2]:
opts = Options()
opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36")

# Conexión con la página web a "scrapear"
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options=opts)
driver.get('https://procesosjudiciales.funcionjudicial.gob.ec/busqueda-filtros')

# Se establece el tipo de proceso a realizar
type_process = int(input("Desea ingresar código para: \n"+
                   "1. Demandante\n"+
                   "2. Demandado\n"))
# Código del demandante o demandado
# Demandante1 = 0968599020001
# Demandante2 = 0992339411001

# Demandado1 = 1791251237001
# Demandado2 = 0968599020001
codigo = input("Ingrese el número de identificación: ")



Desea ingresar código para: 
1. Demandante
2. Demandado
1
Ingrese el número de identificación: 0968599020001


In [3]:
# Me doy cuenta que la pagina carga el formulario dinamicamente luego de que la carga incial ha sido completada
# Por eso tengo que esperar que aparezca 
if type_process == 1:
    input_user = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//input[@formcontrolname = 'cedulaActor']")))
    input_user.send_keys(codigo)
elif type_process == 2:
    input_user = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//input[@formcontrolname = 'cedulaDemandado']")))
    input_user.send_keys(codigo)


# Click al boton buscar
boton_buscar = driver.find_element(By.XPATH,"//button[@type = 'submit']")
boton_buscar.click()


In [4]:
def enviar_proceso(listado_proceso):
    for proceso in listado_proceso:
        id_proceso = proceso.find_element('xpath','.//div[@class="id"]').text
        fecha = proceso.find_element('xpath','.//div[@class="fecha"]').text
        numero_proceso = proceso.find_element('xpath','.//div[@class="numero-proceso"]').text
        accion_infraccion = proceso.find_element('xpath','.//div[@class="accion-infraccion"]').text

        # Se envia la información obtenida a la base de datos
        col.insert_one({
            'id':id_proceso,
            'fecha': fecha,
            'numero_proceso': numero_proceso,
            'accion_infraccion': accion_infraccion
        })

In [12]:
# Se obtiene el número limite para realizar los cambios de página
page_label = driver.find_element(By.XPATH, "//div[@class = 'mat-mdc-paginator-range-label']")
text = page_label.text

# Se utiliza la funcion split para obtener un array por palabra del texto obtenido y luego se obtiene el ultimo registro
parts = text.split()
limit = int(parts[len(parts) - 1])

In [13]:
# Obtengo el listado de procesos esperando antes 10 segundos para dar tiempo de que se carguen por completo
listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))
enviar_proceso(listado_procesos)

# Se obtiene el boton de paso
boton_paso = driver.find_element(By.XPATH,"//button[@aria-label='Página siguiente']")
                                                           
# Se recorren todas las vistas de los registros
for i in range(limit-1):
    try:
        # click al boton de paso
        boton_paso.click()
        # Espero a que se cargue la nueva página
        sleep(random.uniform(8.0,10.0))
        # Obtengo el listado de procesos esperando antes 10 segundos para dar tiempo de que se carguen por completo
        listado_procesos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class = 'causa-individual ng-star-inserted']")))

        enviar_proceso(listado_procesos)
        # Espero a que se cargue la nueva página
        sleep(random.uniform(8.0,10.0))
        # Se obtiene el boton de paso
        boton_paso = driver.find_element(By.XPATH,"//button[@aria-label='Página siguiente']")
    except:
        break

In [27]:
print(second_15)

15
